In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras 

import os
import time
from datetime import datetime

from matplotlib import pyplot as plt
from IPython import display
import numpy as np
from tqdm import tqdm
import cv2

In [4]:
tf.__version__

'2.2.0'

## Create training dataset

In [51]:
USC_FOLDERS = [r"/shared/data1/HDRP_indoor_dataset/png/B", 
               r"/shared/data1/Datasets_from_Noah_team/20200303flarehdrp/png/B",
               r"/shared/data1/Datasets_from_Noah_team/20200313outsidehdrp/png/light/B",
               r"/shared/data1/Datasets_from_Noah_team/20200323/png/B",
               r"/shared/data1/Datasets_from_Noah_team/20200324/png/light/B",
               r"/shared/data1/Datasets_from_Noah_team/20200324/png/no_light/B",
               r"/shared/data1/Datasets_from_Noah_team/20200324_2/png/light/B",
               r"/shared/data1/Datasets_from_Noah_team/20200324_2/png/no_light/B",
              r"/shared/data1/Datasets_from_Noah_team/20200325/png/B",
              r"/shared/data1/Datasets_from_Noah_team/20200330night/png/B",
              r"/shared/data1/Datasets_from_Noah_team/20200331night/png/B",
               r"/shared/data1/Datasets_from_Noah_team/20200402day/png/B",
               r"/shared/data1/Datasets_from_Noah_team/20200402night/png/B",
              r"/shared/data1/Datasets_from_Noah_team/20200407night/png/B",
              r"/shared/data1/Datasets_from_Noah_team/20200409night/png/B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201016/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201020/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201021/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201021_2/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022_2/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022_3/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201023/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201027/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201030/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201102/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201103/2_old_screen_B",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201103_2/2_old_screen_B"]
GT_FOLDERS = [r"/shared/data1/HDRP_indoor_dataset/png/gt", 
              r"/shared/data1/Datasets_from_Noah_team/20200303flarehdrp/png/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200313outsidehdrp/png/light/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200323/png/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200324/png/light/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200324/png/no_light/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200324_2/png/light/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200324_2/png/no_light/gt",
              r"/shared/data1/Datasets_from_Noah_team/20200325/png/gt",
              r"/shared/data1/Datasets_from_Noah_team/20200330night/png/gt",
              r"/shared/data1/Datasets_from_Noah_team/20200331night/png/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200402day/png/gt",
               r"/shared/data1/Datasets_from_Noah_team/20200402night/png/gt",
             r"/shared/data1/Datasets_from_Noah_team/20200407night/png/gt",
              r"/shared/data1/Datasets_from_Noah_team/20200409night/png/gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201016/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201020/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201021/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201021_2/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022_2/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201022_3/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201023/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201027/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201030/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201102/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201103/1_gt",
              r"/shared/data1/datasets_04_11_2020/capturing_datasets/20201103_2/1_gt"]

In [52]:
from train_utils.utils  import dataset, image_io, image_transform
from train_utils.train  import losses, train_step

In [53]:
png_image_reader = image_io.ImageReaderPNG(transform_list=None)

In [54]:
usc_arrays = dataset.get_images_array(folders_list=USC_FOLDERS, image_reader=png_image_reader, images_extension='.png')

100%|██████████| 34/34 [00:09<00:00,  3.49it/s]


In [55]:
len(usc_arrays)

1105

In [56]:
gt_arrays = dataset.get_images_array(folders_list=GT_FOLDERS, image_reader=png_image_reader, images_extension='.png')

100%|██████████| 34/34 [00:09<00:00,  3.45it/s]


In [57]:
len(gt_arrays)

1105

In [58]:
from skimage.transform import rescale, resize

In [59]:
def img_resize(image):
    new_img = cv2.resize(image, (3264//8, 2448//8))#, anti_aliasing=True)
    #print('resized')
    return new_img

In [60]:
usc_arrays_downscaled = [img_resize(image) for image in tqdm(usc_arrays)]

100%|██████████| 1105/1105 [00:00<00:00, 1218.40it/s]


In [61]:
gt_arrays_downscaled = [img_resize(image) for image in tqdm(gt_arrays)]

100%|██████████| 1105/1105 [00:00<00:00, 1185.63it/s]


In [62]:
np.savez("usc_arrays_downscaled_capturing", usc_arrays_downscaled)
np.savez("gt_arrays_downscaled_capturing", gt_arrays_downscaled)

## GPU selection

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[2], True) 
tf.config.experimental.set_visible_devices(physical_devices[2], 'GPU')

In [7]:
tf.keras.backend.clear_session()

## Dataset creation

In [63]:
usc_arrays_downscaled = np.load("/home/p00536919/Flare_removal/Downscale_ref/usc_arrays_downscaled_capturing.npz")['arr_0']
gt_arrays_downscaled = np.load("/home/p00536919/Flare_removal/Downscale_ref/gt_arrays_downscaled_capturing.npz")['arr_0']

In [64]:
from guided_filter_tf.guided_filter import guided_filter

for num, (usc_image, gt_image) in tqdm(list(enumerate(zip(usc_arrays_downscaled, gt_arrays_downscaled)))):
    
    gt_image = guided_filter(x=tf.cast(gt_image[np.newaxis, :, :, :], tf.float32),
                             y=tf.cast(usc_image[np.newaxis, :, :, :], tf.float32), r=100, nhwc=True)
    gt_image = gt_image.numpy()[0]
    gt_arrays_downscaled[num] = gt_image

100%|██████████| 1105/1105 [00:14<00:00, 76.41it/s]


In [65]:
training_dataset = (usc_arrays_downscaled, gt_arrays_downscaled)

In [66]:
train_data_generator = dataset.DataGenerator(training_dataset, patch_size=224, batch_size=1, image_size=(2448//8, 3264//8), shuffle=True, pad=2)

### Clear logs

In [16]:
#!RMDIR /Q /S G:\logs\flare_removal
#!rm -rf /shared/p00536919/logs/flare_removal

## Build models

### Generator

In [12]:
def conv_block(growth_rate, filters, kernel_size, strides, x):
    x = tf.keras.layers.Conv2D(growth_rate * filters, kernel_size, padding='same', strides=strides, data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def dilated_conv_block(growth_rate, filters, kernel_size, dilation_rate, x):
    x = tf.keras.layers.Conv2D(growth_rate * filters, kernel_size, padding='same', dilation_rate=dilation_rate, data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def conv_skip_block(growth_rate, filters, kernel_size, x):
    x = tf.keras.layers.Conv2DTranspose(growth_rate * filters, kernel_size, padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x


def deconv_block(growth_rate, filters, kernel_size, strides, x):
    x = tf.keras.layers.Conv2DTranspose(growth_rate * filters, kernel_size, padding='same', strides=strides, data_format='channels_first')(x)
    x = tf.keras.layers.AveragePooling2D((2, 2), 1, padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    return x

In [13]:
def Generator(height=None, width=None, input_channels=3, filters=8):

    inputs = tf.keras.Input(shape=[input_channels, height, width])

    x = conv_block(growth_rate=2, filters=filters, kernel_size=(5, 5), strides=1, x=inputs)
    res1 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    res2 = x

    x = conv_block(growth_rate=4, filters=filters, kernel_size=(3, 3), strides=2, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=2, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=4, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=8, x=x)
    x = dilated_conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), dilation_rate=16, x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=4, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res2])
    x = conv_skip_block(growth_rate=4, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=8, filters=filters, kernel_size=(3, 3), strides=1, x=x)

    x = deconv_block(growth_rate=2, filters=filters, kernel_size=(4, 4), strides=2, x=x)

    x = tf.keras.layers.Concatenate(axis=1)([x, res1])
    x = conv_skip_block(growth_rate=2, filters=filters, kernel_size=(1, 1), x=x)

    x = conv_block(growth_rate=1, filters=filters, kernel_size=(3, 3), strides=1, x=x)
    x = tf.keras.layers.Conv2D(3, (3, 3), padding='same', data_format='channels_first')(x)
    x = tf.keras.layers.Subtract()([inputs, x])
    
    _model = tf.keras.Model(inputs=inputs, outputs=x, name='derain_net')
    return _model  


In [14]:
generator = Generator()

In [ ]:
generator.summary()

In [45]:
generator.save('deflare.h5')

In [46]:
import netron

In [47]:
netron.start('deflare.h5', port=8989)

Serving 'deflare.h5' at http://localhost:8989


In [48]:
netron.stop(port=8989)


Stopping http://localhost:8989


## Define losses

### Generator loss

In [15]:
from train_utils.train  import losses

In [16]:
vgg16_loss_1_3 = losses.make_VGG16_loss(blocks_dict={1:1, 2:1, 3:1, 4:1, 5:1}, 
                                          weights_path=r"/home/p00536919/usc-image-enhancement/srs_refactor/refactored_pipeline/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                          loss_type='MSE')

In [17]:
def generator_loss(gen_output, target):
    
    gen_output = tf.transpose(gen_output, [0,2,3,1])
    target = tf.transpose(target, [0,2,3,1])

    l1_loss_val = losses.L1_loss(target, gen_output)
    
    #ms_ssim_loss_val = losses.MS_SSIM_loss(target, gen_output)
    ms_ssim_loss_val = (1 - tf.math.reduce_mean(tf.image.ssim_multiscale(target, gen_output, 1, power_factors=(0.4, 0.25, 0.25, 0.2363, 0.1333))))

    
    vgg_loss_val = vgg16_loss_1_3(target, gen_output)

    total_gen_loss_val = 0.5 * l1_loss_val + 1 * ms_ssim_loss_val  + 0.08 * vgg_loss_val

    return total_gen_loss_val, [l1_loss_val, ms_ssim_loss_val, vgg_loss_val]

## Define the Optimizers and Checkpoint-saver

In [26]:
"""decay_steps = 1000
initial_learning_rate_generator = 2e-4
initial_learning_rate_discriminator = 1e-4

step_decay = 1e-2

boundaries = [1000]

step_decay_gen = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, [initial_learning_rate_generator, initial_learning_rate_generator*step_decay])"""

'decay_steps = 1000\ninitial_learning_rate_generator = 2e-4\ninitial_learning_rate_discriminator = 1e-4\n\nstep_decay = 1e-2\n\nboundaries = [1000]\n\nstep_decay_gen = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, [initial_learning_rate_generator, initial_learning_rate_generator*step_decay])'

In [18]:
#generator_optimizer = tf.keras.optimizers.Adam(step_decay_gen, beta_1=0.5)
import tensorflow_addons as tfa

radam = tfa.optimizers.RectifiedAdam(lr=2e-4)
generator_optimizer = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

In [19]:
checkpoint_dir = '/shared/p00536919/training_checkpoints/flare_removal/' + datetime.now().strftime("%Y%m%d-%H%M%S")+"_capturing"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer, generator=generator)

## Training

In [20]:
log_dir=r"/shared/p00536919/logs/flare_removal/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")+"_capturing")

file_writer_img = tf.summary.create_file_writer(log_dir + "fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")+"_capturing" + '/img')

In [70]:
from train_utils.train  import train_step 

In [71]:
train_step = train_step.make_train_step()

In [72]:
import albumentations as A

In [73]:
transform = A.Compose(
    [A.VerticalFlip(p=0.5), 
     A.HorizontalFlip(p=0.5), 
     A.RandomScale (scale_limit=(1,2), interpolation=1, always_apply=False, p=0.5),
     A.RandomCrop(224, 224, always_apply=False, p=1.0), 
     A.ChannelShuffle(p=0.5)],
    additional_targets={'image0': 'image'}
)

In [74]:
def fit(epochs, run):
           
    for epoch in range(epochs):  
        epoch += int(epochs * run)
        start = time.time()
        display.clear_output(wait=True)
        
        if (epoch+1) % 50 == 0:
            generator.save_weights("log_weights0_capturing.hdf5")
            with tf.device('/GPU:0'):
                tf.keras.backend.clear_session()
                test_generator = Generator()
                test_generator.load_weights('log_weights0_capturing.hdf5')                
                #test_image = np.zeros((2448, 3264, 6))
                ing_index = np.random.randint(len(usc_arrays_downscaled))
                test_image = usc_arrays_downscaled[ing_index][np.newaxis, :264, :480, :]
                test_image = tf.cast(test_image, tf.float32)
                #test_image = tf.image.resize(test_image, [256,256])
                preds = test_generator(tf.transpose(test_image,[0,3,1,2]), training=True)  
                preds = tf.transpose(preds, [0,2,3,1])
            
            with file_writer_img.as_default():                
                tf.summary.image("Input_image", usc_arrays_downscaled[ing_index][np.newaxis, :264, :480, :3], step=epoch)
                tf.summary.image("Ground truth", gt_arrays_downscaled[ing_index][np.newaxis, :264, :480, :3], step=epoch)
                tf.summary.image("Model results", preds.numpy()[:, :,:,:3], step=epoch)
                
                
        print("Epoch: ", epoch)
        train_data_generator.on_epoch_end()
        # Train
        gen_total_loss, l1_loss_val, ms_ssim_loss_val, vgg_loss_val  = 0, 0, 0, 0
        
        for n, (input_image, target) in enumerate(train_data_generator):
            print('.', end='')
            if (n+1) % 100 == 0:
                print('\n')
            input_image = np.array(input_image)            
            target = np.array(target)
            transformed = transform(image=input_image[0], image0=target[0])
            
            input_image = transformed['image'][np.newaxis,...]
            target = transformed['image0'][np.newaxis,...]
            
            input_image = tf.cast(input_image, tf.float32)
            target = tf.cast(target, tf.float32)
            losses = train_step(input_image=tf.transpose(tf.cast(input_image, tf.float32), [0,3,1,2]),
                                    target=tf.transpose(target,[0,3,1,2]),
                                    generator=generator,                                    
                                    generator_loss=generator_loss,
                                    generator_optimizer=generator_optimizer,
                                    training=True)
            
            gen_total_loss += losses[0]
            
            l1_loss_val += losses[1][0]
            ms_ssim_loss_val += losses[1][1]
            vgg_loss_val += losses[1][2]
            
            
            
        print()
        with summary_writer.as_default():
            tf.summary.scalar('gen_total_loss', gen_total_loss/len(train_data_generator), step=epoch)
            #tf.summary.scalar('gan_loss_val', gan_loss_val/len(train_data_generator), step=epoch)
            tf.summary.scalar('l1_loss_val', l1_loss_val/len(train_data_generator), step=epoch)
            tf.summary.scalar('ms_ssim_loss_val', ms_ssim_loss_val/len(train_data_generator), step=epoch)
            tf.summary.scalar('vgg_loss_val', vgg_loss_val/len(train_data_generator), step=epoch)
            #tf.summary.scalar('vgg_loss_val', vgg_loss_val/len(train_data_generator), step=epoch)
            #tf.summary.scalar('disc_loss ', disc_loss /len(train_data_generator), step=epoch)
            
        # saving (checkpoint) the model every 250 epochs
        if (epoch + 1) % 100 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            
        """if (epoch + 1) % 1000 == 0:
            generator.save_weights("avg_weights_gen_{}-1.hdf5".format(epoch))
            #discriminator.save_weights("avg_weights_disc_{}-1.hdf5".format(epoch))"""
            
        print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                                time.time()-start))


In [ ]:
for i in range(100):
    fit(EPOCHS, i)

Epoch:  3562
....................................................................................................

....................................................................................................

....................................................................................................

....................................................................................................

....................................................................................................

....................................................................................................

....................................................................................................

....................................................................................................

..........

### Restore from checkpoint

In [69]:
checkpoint.restore("/shared/p00536919/training_checkpoints/flare_removal/20200114-114537/ckpt-10")

### Save Models

In [51]:
generator.save_weights(r'./weights/trained_vanilla_pix2pix_hist_superlight_no_rescale_FINAL_gen_no_gamma_11_5_2020.hdf5')
discriminator.save_weights(r'./weights/trained_vanilla_pix2pix_hist_superlight_no_rescale_FINAL_disc_no_gamma_18_5_2020.hdf5')

In [39]:
generator.load_weights(r'F:\models\new_approach_gen_27_2_2020.hdf5')
discriminator.load_weights(r'F:\models\new_approach_disc_27_2_2020.hdf5')

In [68]:
plt.imsave(r"usc1.png", usc_arrays_downscaled[495][:296, :408, :])

In [69]:
plt.imsave(r"gt1.png", gt_arrays_downscaled[495][:296, :408, :])

In [72]:
test_image = usc_arrays_downscaled[495][np.newaxis, :296, :408, :]
test_image = tf.cast(test_image, tf.float32)
#test_image = tf.image.resize(test_image, [256,256])
preds = generator(tf.transpose(test_image,[0,3,1,2]), training=True)  
preds = tf.transpose(preds, [0,2,3,1])
plt.imsave(r"preds1.png", np.clip(preds.numpy(),0,1)[0])

In [74]:
tf.__version__

'2.2.0'